In [ ]:
#
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
start_of_train_year=2015
end_of_train_year=2019



In [ ]:
#demand
demand_folder=data_and_output_dir+'output/demand_aggreate' 
#each are store in os.path.join(output_folder_path, f'aggregated_demand_population_{year}.csv')

#weather
weather_folder = data_and_output_dir +'output/'+'historical_weather'
# each file store in format of os.path.join(folder_to_store, f"{rb}_WRF_Hourly_Mean_Meteorology_{year}.csv")

output_folder=data_and_output_dir +'output/'+'demand_weather_combine'

In [ ]:
#Trash 


#sample
print()